# ***Block Deep Neural Network-Based Signal Detector for Generalized Spatial Modulation / Imperfect Channel Estimation (Simulations)***

This code is the simulations of the model studied in the paper "Block Deep Neural Network-Based Signal Detector for Generalized Spatial Modulation" with the presence of imperfect channel estimation.

***Libraries***

In [ ]:
# Enter the path of the "Block-DNN" folder in order to save and load the files
B_DNN_folder_path = ""

from tensorflow.keras import models
import matplotlib.pyplot as plt
import numpy as np
import WirelessCommLib as wcl
import pickle

***Functions***

In [ ]:
# =====================================================================================================
# 1. Simulation
#
# ARGUMENTS
# 1-) parameter_dict: Dictionary of GSM parameters (Data Type: dict)
# 2-) detector_type: Detector Type (Data Type: str | Condition: "B-DNN", "B-DNN+", "ML", "B-ZF" or "B-MMSE")
#
# OUTPUT
# - output_dict: Dictionary of simulation results (Data Type: dict)
# =====================================================================================================
def Simulation(parameter_dict, detector_type):
    # GSM PARAMETERS
    SNRdB_array = parameter_dict["SNRdB_array"]
    Ns = parameter_dict["Ns"] # Number of simulation time slots
    Nt = parameter_dict["Nt"] # Number of transmit antennas
    Np = parameter_dict["Np"] # Number of active transmit antennas
    Nr = parameter_dict["Nr"] # Number of receive antennas
    M = parameter_dict["M"] # Constellation size
    mod_type = parameter_dict["mod_type"] # Modulation type
    beta = parameter_dict["beta"] # Channel estimation error power

    N_tot = wcl.Combination(Nt, Np) # Number of total transmit antenna combinations (TACs)
    ns = int(np.floor(np.log2(N_tot))) # Number of spatial bits transmitted during a time-slot
    m = int(np.log2(M)) # Number of information bits transmitted from a single active antenna during a time-slot
    ni = Np * m # Number of total information bits transmitted during a time-slot
    n_tot = ns + ni # Number of total bits transmitted during a time-slot
    N = 2 ** ns # Number of illegitimate TACs
    P_tot = Np # Total transmit power
    is_normalized = True
    ss = wcl.Constellation(M, mod_type, is_normalized) # Signal set
    TAC_set = wcl.OptimalTAC_Set(Nt, Np, N) # Optimal TAC set

    # DEEP LEARNING MODEL
    if detector_type == "B-DNN":
        model_path = B_DNN_folder_path + "/Trained Models/"
        model_name = "B_DNN_model_Np" + str(Np) + "_Nr" + str(Nr) + "_M" + str(M) + mod_type + "_SFVG" + ".h5"
    elif detector_type == "B-DNN+":
        model_path = B_DNN_folder_path + "/Trained Models/"
        model_name = "B_DNN+_model_Np" + str(Np) + "_Nr" + str(Nr) + "_M" + str(M) + mod_type + "_SFVG" + ".h5"

    # SIMULATION
    num_error_array = np.zeros((len(SNRdB_array),))
    SNRdB_index = 0
    print(detector_type + " Detector, SNR (dB) =", end=" ")
    for SNRdB in SNRdB_array:
        print(str(SNRdB), end=" ")
        N0 = P_tot / (10 ** (SNRdB / 10)) # Noise power
        num_error = 0
        for j in range(Ns):
            bit_array = np.random.randint(2, size=(n_tot,))
            x = wcl.EncodeBits(bit_array, ss, TAC_set, ns, m, Nt, Np) # Transmitted vector
            H_est = wcl.Channel([Nr, Nt]) # Estimated Rayleigh fading channel
            H_err = wcl.Channel([Nr, Nt]) # Erroneous Rayleigh fading channel
            H = np.sqrt(1 - beta) * H_est + np.sqrt(beta) * H_err
            n = wcl.AWGN(N0, [Nr, 1]) # AWGN
            y = np.matmul(H, x) + n # Received vector

            if detector_type == "B-DNN" or detector_type == "B-DNN+":
                detected_bit_array = B_DNN_Detector(B_DNN_model, TAC_set, ss, ns, m, y, H_est)
            elif detector_type == "ML":
                detected_bit_array = ML_Detector(TAC_set, ss, ns, m, y, H_est)
            elif detector_type == "B-ZF":
                detected_bit_array = B_ZF_Detector(TAC_set, ss, ns, m, y, H_est)
            elif detector_type == "B-MMSE":
                detected_bit_array = B_MMSE_Detector(TAC_set, ss, ns, m, y, H_est, N0)

            num_error = num_error + np.logical_xor(bit_array, detected_bit_array).sum()

        num_error_array[SNRdB_index] = num_error
        SNRdB_index = SNRdB_index + 1

    print()
    BER_array = num_error_array / (Ns * n_tot)
    output_dict = {"SNRdB_array" : SNRdB_array, "BER_array" : BER_array, "num_error_array" : num_error_array}
    return output_dict
# =====================================================================================================


# =====================================================================================================
# 2. Block DNN Detector
#
# ARGUMENTS
# 1-) model: Deep learning model (Data Type: tensorflow.python.keras.engine.functional.Functional)
# 2-) TAC_set: Optimal TAC set (Data Type: numpy.ndarray | Shape: (N, Np))
# 3-) ss: Signal set (Data Type: numpy.ndarray | Shape: (1, M))
# 4-) ns: Number of spatial bits transmitted during a time-slot (Data Type: int)
# 5-) m: Number of information bits transmitted from a single active antenna during a time-slot (Data Type: int)
# 6-) y: Received signal vector (Data Type: numpy.ndarray | Shape: (Nr, 1))
# 7-) H: Rayleigh fading channel (Data Type: numpy.ndarray | Shape: (Nr, Nt))
#
# OUTPUT
# - detected_bit_array: Bit array obtained by B-DNN detector (Data Type: numpy.ndarray or list | Shape: (n_tot,))
# =====================================================================================================
def B_DNN_Detector(model, TAC_set, ss, ns, m, y, H):
    N = TAC_set.shape[0]
    Np = TAC_set.shape[1]

    distance_array = np.zeros((N,))
    s_dict_array = []
    for TAC_index in range(N):
        TAC = TAC_set[TAC_index, :]
        H_active = H[:, TAC - 1]
        d = np.transpose(np.concatenate((wcl.FVG(y, "SFVG"), wcl.FVG(H_active, "SFVG"))))
        prediction = model.predict(d)
        s_dict = Pred2Dict(ss, prediction)
        s_dict_array.append(s_dict)
        s_vec = s_dict["s_vec"]
        distance_array[TAC_index] = np.linalg.norm(y - np.matmul(H_active, s_vec)) ** 2

    detected_TAC_index = np.argmin(distance_array)
    detected_s_dict = s_dict_array[detected_TAC_index]
    predicted_symbol_index_vec = detected_s_dict["predicted_symbol_index_vec"]

    n_tot = ns + Np * m
    detected_bit_array = np.zeros([n_tot,])
    detected_bit_array[0 : ns] = wcl.Dec2Bin(detected_TAC_index, ns)
    for active_antenna_index in range(Np):
        detected_symbol_bits = wcl.Dec2Bin(predicted_symbol_index_vec[active_antenna_index], m)

        start_bit_index = ns + active_antenna_index * m
        stop_bit_index = ns + (active_antenna_index + 1) * m
        detected_bit_array[start_bit_index : stop_bit_index] = detected_symbol_bits

    return detected_bit_array
# =====================================================================================================


# =====================================================================================================
# 3. Conversion From Model Predictions To Dictionary Consisting Of Symbols & Symbol Indices
#
# ARGUMENTS
# 1-) ss: Signal set (Data Type: numpy.ndarray | Shape: (1, M))
# 2-) prediction: Model prediction (Data Type: list | Shape: (Np,))
#
# OUTPUT
# - s_dict: Dictionary of predicted symbol vector and symbol indices (Data Type: dict)
# =====================================================================================================
def Pred2Dict(ss, prediction):
    Np = len(prediction)
    s_dict = {"s_vec" : np.zeros((Np, 1), dtype=np.complex64), "predicted_symbol_index_vec" : np.zeros((Np,))}
    for active_antenna_index in range(Np):
        current_prediction = prediction[active_antenna_index]
        predicted_symbol_index = np.argmax(current_prediction)
        s_dict["predicted_symbol_index_vec"][active_antenna_index] = predicted_symbol_index
        s_dict["s_vec"][active_antenna_index, 0] = ss[0, predicted_symbol_index]
    return s_dict
# =====================================================================================================


# =====================================================================================================
# 4. Maximum Likelihood (ML) Detector
# 
# ARGUMENTS
# 1-) TAC_set = Optimal TAC set (Data Type: numpy.ndarray | Shape: (N, Np))
# 2-) ss: Signal set (Data Type: numpy.ndarray | Shape: (1, M))
# 3-) ns: Number of spatial bits transmitted during a time-slot (Data Type: int)
# 4-) m: Number of information bits transmitted from a single active antenna during a time-slot (Data Type: int)
# 5-) y: Received signal vector (Data Type: numpy.ndarray | Shape: (Nr, 1))
# 6-) H: Rayleigh fading channel (Data Type: numpy.ndarray | Shape: (Nr, Nt))
#
# OUTPUT
# - detected_bit_array: Bit array obtained by ML detector (Data Type: numpy.ndarray or list | Shape: (n_tot,))
# =====================================================================================================
def ML_Detector(TAC_set, ss, ns, m, y, H):
    N = TAC_set.shape[0]
    Np = TAC_set.shape[1]
    M = 2 ** m
    num_symbol_comb = M ** Np
    num_total_comb = N * num_symbol_comb

    total_comb_index = 0
    distance_matrix = np.zeros((num_total_comb, Np + 2))
    for TAC_index in range(N):
        TAC = TAC_set[TAC_index, :]
        H_active = H[:, TAC - 1]
        for symbol_comb_index in range(num_symbol_comb):
            if symbol_comb_index == 0:
                symbol_comb_str = np.base_repr(symbol_comb_index, M, Np)
            else:
                symbol_comb_str = np.base_repr(symbol_comb_index, M, Np - len(np.base_repr(symbol_comb_index, M)))
            symbol_comb = list(map(int, symbol_comb_str))
            s_vec = ss[0, symbol_comb].reshape(Np, 1)
            distance = np.linalg.norm(y - np.matmul(H_active, s_vec)) ** 2

            distance_matrix[total_comb_index, 0] = TAC_index
            distance_matrix[total_comb_index, 1 : Np + 1] = symbol_comb
            distance_matrix[total_comb_index, Np + 1] = distance
            total_comb_index = total_comb_index + 1
  
    distance_array = distance_matrix[:, Np + 1]
    min_distance_index = np.argmin(distance_array)
    detected_TAC_index = distance_matrix[min_distance_index, 0]
    detected_symbol_index_vec = distance_matrix[min_distance_index, 1 : Np + 1]

    n_tot = ns + Np * m
    detected_bit_array = np.zeros([n_tot,])
    detected_bit_array[0 : ns] = wcl.Dec2Bin(detected_TAC_index, ns)
    for active_antenna_index in range(Np):
        detected_symbol_bits = wcl.Dec2Bin(detected_symbol_index_vec[active_antenna_index], m)

        start_bit_index = ns + active_antenna_index * m
        stop_bit_index = ns + (active_antenna_index + 1) * m
        detected_bit_array[start_bit_index : stop_bit_index] = detected_symbol_bits
  
    return detected_bit_array
# =====================================================================================================


# =====================================================================================================
# 5. Block Zero Forcing (B-ZF) Detector
# 
# ARGUMENTS
# 1-) TAC_set = Optimal TAC set (Data Type: numpy.ndarray | Shape: (N, Np))
# 2-) ss: Signal set (Data Type: numpy.ndarray | Shape: (1, M))
# 3-) ns: Number of spatial bits transmitted during a time-slot (Data Type: int)
# 4-) m: Number of information bits transmitted from a single active antenna during a time-slot (Data Type: int)
# 5-) y: Received signal vector (Data Type: numpy.ndarray | Shape: (Nr, 1))
# 6-) H: Rayleigh fading channel (Data Type: numpy.ndarray | Shape: (Nr, Nt))
#
# OUTPUT
# - detected_bit_array: Bit array obtained by ML detector (Data Type: numpy.ndarray or list | Shape: (n_tot,))
# =====================================================================================================
def B_ZF_Detector(TAC_set, ss, ns, m, y, H):
    N = TAC_set.shape[0]
    Np = TAC_set.shape[1]
    M = 2 ** m

    distance_array = np.zeros((N,))
    s_vec_array = []
    for TAC_index in range(N):
        TAC = TAC_set[TAC_index, :]
        H_active = H[:, TAC - 1]
        s_vec_hat = np.matmul(np.linalg.pinv(H_active), y)
        s_vec = ss[0, np.argmin(np.abs(s_vec_hat - ss), axis=1)].reshape(Np, 1)
        s_vec_array.append(s_vec)
        distance_array[TAC_index] = np.linalg.norm(y - np.matmul(H_active, s_vec)) ** 2
  
    detected_TAC_index = np.argmin(distance_array)
    s_vec = s_vec_array[detected_TAC_index]

    n_tot = ns + Np * m
    detected_bit_array = np.zeros([n_tot,])
    detected_bit_array[0 : ns] = wcl.Dec2Bin(detected_TAC_index, ns)
    for active_antenna_index in range(Np):
        symbol = s_vec[active_antenna_index, 0]
        detected_symbol_index = np.argmin(np.abs(symbol - ss))
        detected_symbol_bits = wcl.Dec2Bin(detected_symbol_index, m)

        start_bit_index = ns + active_antenna_index * m
        stop_bit_index = ns + (active_antenna_index + 1) * m
        detected_bit_array[start_bit_index : stop_bit_index] = detected_symbol_bits
  
    return detected_bit_array
# =====================================================================================================


# =====================================================================================================
# 6. Block Minimum Mean Squared Error (B-MMSE) Detector
# 
# ARGUMENTS
# 1-) TAC_set = Optimal TAC set (Data Type: numpy.ndarray | Shape: (N, Np))
# 2-) ss: Signal set (Data Type: numpy.ndarray | Shape: (1, M))
# 3-) ns: Number of spatial bits transmitted during a time-slot (Data Type: int)
# 4-) m: Number of information bits transmitted from a single active antenna during a time-slot (Data Type: int)
# 5-) y: Received signal vector (Data Type: numpy.ndarray | Shape: (Nr, 1))
# 6-) H: Rayleigh fading channel (Data Type: numpy.ndarray | Shape: (Nr, Nt))
# 7-) N0: Noise power (Data Type: float)
#
# OUTPUT
# - detected_bit_array: Bit array obtained by ML detector (Data Type: numpy.ndarray or list | Shape: (n_tot,))
# =====================================================================================================
def B_MMSE_Detector(TAC_set, ss, ns, m, y, H, N0):
    N = TAC_set.shape[0]
    Np = TAC_set.shape[1]
    M = 2 ** m

    distance_array = np.zeros((N,))
    s_vec_array = []
    for TAC_index in range(N):
        TAC = TAC_set[TAC_index, :]
        H_active = H[:, TAC - 1]
        inv_component = np.linalg.inv(np.matmul(wcl.Hermitian(H_active), H_active) + N0 * np.eye(Np))
        s_vec_hat = np.matmul(np.matmul(inv_component, wcl.Hermitian(H_active)), y)
        s_vec = ss[0, np.argmin(np.abs(s_vec_hat - ss), axis=1)].reshape(Np, 1)
        s_vec_array.append(s_vec)
        distance_array[TAC_index] = np.linalg.norm(y - np.matmul(H_active, s_vec)) ** 2
  
    detected_TAC_index = np.argmin(distance_array)
    s_vec = s_vec_array[detected_TAC_index]
    n_tot = ns + Np * m
    detected_bit_array = np.zeros([n_tot,])
    detected_bit_array[0 : ns] = wcl.Dec2Bin(detected_TAC_index, ns)
    for active_antenna_index in range(Np):
        symbol = s_vec[active_antenna_index, 0]
        detected_symbol_index = np.argmin(np.abs(symbol - ss))
        detected_symbol_bits = wcl.Dec2Bin(detected_symbol_index, m)

        start_bit_index = ns + active_antenna_index * m
        stop_bit_index = ns + (active_antenna_index + 1) * m
        detected_bit_array[start_bit_index : stop_bit_index] = detected_symbol_bits
  
    return detected_bit_array
# =====================================================================================================

***GSM Parameters***

In [ ]:
fig_name = "PerfectCSI"
if fig_name == "PerfectCSI":
    parameter_dict = {"SNRdB_array" : np.arange(0, 25, 4), "Ns" : 100000, "Nt" : 4, "Np" : 2, "Nr" : 2, "M" : 2, "mod_type" : "PSK", "beta" : 0}
elif fig_name == "ImperfectCSI_beta0.001":
    parameter_dict = {"SNRdB_array" : np.arange(0, 25, 4), "Ns" : 100000, "Nt" : 4, "Np" : 2, "Nr" : 2, "M" : 2, "mod_type" : "PSK", "beta" : 0.001}
elif fig_name == "ImperfectCSI_beta0.01":
    parameter_dict = {"SNRdB_array" : np.arange(0, 25, 4), "Ns" : 100000, "Nt" : 4, "Np" : 2, "Nr" : 2, "M" : 2, "mod_type" : "PSK", "beta" : 0.01}
elif fig_name == "ImperfectCSI_beta0.1":
    parameter_dict = {"SNRdB_array" : np.arange(0, 25, 4), "Ns" : 100000, "Nt" : 4, "Np" : 2, "Nr" : 2, "M" : 2, "mod_type" : "PSK", "beta" : 0.1}
else:
    parameter_dict = { } # Create your own parameter dictionary keeping the format above

***Simulations***

In [ ]:
B_DNN_output_dict = Simulation(parameter_dict, "B-DNN")
B_DNN_plus_output_dict = Simulation(parameter_dict, "B-DNN+")
ML_output_dict = Simulation(parameter_dict, "ML")
B_ZF_output_dict = Simulation(parameter_dict, "B-ZF")
B_MMSE_output_dict = Simulation(parameter_dict, "B-MMSE")

results_dict = {"B_DNN_output_dict" : B_DNN_output_dict,
                "B_DNN+_output_dict" : B_DNN_plus_output_dict,
                "ML_output_dict" : ML_output_dict,
                "B_ZF_output_dict" : B_ZF_output_dict,
                "B_MMSE_output_dict" : B_MMSE_output_dict}

fig_path = B_DNN_folder_path + "/Results/ImperfectCSI/"
file_name = fig_name + "_results.pkl"
save_file = open(file_name, "wb")
pickle.dump(results_dict, save_file)
save_file.close()

***Plots***

In [ ]:
# User Specifications///////////////////////////////////////////////////////////
fig_name = "PerfectCSI" # Enter the figure name that you want to plot.
fig_title = "Perfect CSI" # Enter the title of the figure.
# //////////////////////////////////////////////////////////////////////////////

fig_path = B_DNN_folder_path + "/Results/ImperfectCSI/"
file_name = fig_name + "_results.pkl"
save_file = open(file_name, "rb")
results_dict = pickle.load(save_file)

B_DNN_output_dict = results_dict["B_DNN_output_dict"]
B_DNN_plus_output_dict = results_dict["B_DNN+_output_dict"]
ML_output_dict = results_dict["ML_output_dict"]
B_ZF_output_dict = results_dict["B_ZF_output_dict"]
B_MMSE_output_dict = results_dict["B_MMSE_output_dict"]

plot_B_DNN, = plt.semilogy(B_DNN_output_dict["SNRdB_array"], B_DNN_output_dict["BER_array"], "g-*", lw=2.5, markersize=10, label="B-DNN")
plot_B_DNN_plus, = plt.semilogy(B_DNN_plus_output_dict["SNRdB_array"], B_DNN_plus_output_dict["BER_array"], "y-s", lw=2.5, markersize=10, label="B-DNN+")
plot_ML, = plt.semilogy(ML_output_dict["SNRdB_array"], ML_output_dict["BER_array"], "r-o", lw=2.5, markersize=10, label="ML")
plot_B_ZF, = plt.semilogy(B_ZF_output_dict["SNRdB_array"], B_ZF_output_dict["BER_array"], "b-d", lw=2.5, markersize=10, label="B-ZF")
plot_B_MMSE, = plt.semilogy(B_MMSE_output_dict["SNRdB_array"], B_MMSE_output_dict["BER_array"], "m-v", lw=2.5, markersize=10, label="B-MMSE")
plt.legend(handles=[plot_B_DNN, plot_B_DNN_plus, plot_ML, plot_B_ZF, plot_B_MMSE])
plt.title(fig_title, fontsize=15)
plt.xlabel("SNR (dB)", fontsize=15)
plt.ylabel("BER", fontsize=15)
plt.xlim([0, 24])
plt.ylim([1e-3, 1])
plt.grid()
plt.show()